# Ira pegar os dados de futebol virtual dos ultimos 90 dias

In [37]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import bs4, time
from datetime import datetime, timedelta
import pandas as pd
pd.options.display.max_rows = None

In [20]:
driver = webdriver.Chrome(executable_path='./webdriver/linux/chromedriver')

In [5]:
# DIA DE HOJE
d2 = datetime.strptime('2020-04-20', '%Y-%m-%d')

# DIA LIMITE DE DADOS NA BETFAIR
d1 = datetime.strptime('2020-01-21', '%Y-%m-%d')

# Calculo da quantidade de dias
quantidade_dias = abs((d2 - d1).days)

# TALVEZ SO TENHA DADOS DE 90 DIAS, SE ISSO FOR VERDADE A DATA LIMITE MUDARA E NAO SERA DIA 21 DE JANEIRO

In [6]:
event_data = {'date': '', 'time': '', 'team_home': '', 'score': '', 'team_back': ''}

In [7]:
data = pd.DataFrame(columns=list(event_data.keys()), dtype=str)
saved_days = []

In [25]:
for i in range(quantidade_dias):
    day = '20' + (datetime.today() - timedelta(days=i+1)).strftime('%y-%m-%d')
    
    
    if day not in saved_days:
        link = 'https://www.betfair.com/sport/virtuals-results?sport=SOCCER&day=' + day
        driver.get(link)

        total = 0
        count = 0
        while total != '475':
            total = driver.find_element_by_class_name('number-of-results').text
            driver.find_element_by_tag_name('html').send_keys(Keys.END)
            time.sleep(1)
            driver.find_element_by_tag_name('html').send_keys(Keys.HOME)
            count +=1
            if count >=20:
                print('\ndia com menos de 475\n')
                next_or_break = input('1 para parar de tentar pegar os dados ou outra tecla para continuar\n:')
                if str(next_or_break) == '1':
                    break

        htmlPage = bs4.BeautifulSoup(driver.page_source, features='html.parser')
        table_results = htmlPage.find('div',{'class':'result-list-wrapper'})

        data_day = []
        for event in table_results.find_all('div',{'class':'result-title'}):
            content = event.text.split('\n')
            event_data = {'date': '', 'time': '', 'team_home': '', 'score': '', 'team_back': ''}

            event_data['date'] = day
            event_data['time'] = content[1]
            event_data['team_home'] = content[2]
            event_data['score'] = content[4]
            event_data['team_back'] = content[6]

            data_day.append(event_data)

        df = pd.DataFrame(data_day, columns=list(event_data.keys()), dtype=str)

        data = data.append(df)

        saved_days.append(day)
        print('{} saved day: {}'.format(i, day))
#         if i == 2:
#             break

16 saved day: 2020-04-03
17 saved day: 2020-04-02
18 saved day: 2020-04-01
19 saved day: 2020-03-31
20 saved day: 2020-03-30
21 saved day: 2020-03-29
22 saved day: 2020-03-28
23 saved day: 2020-03-27
24 saved day: 2020-03-26
25 saved day: 2020-03-25
26 saved day: 2020-03-24
27 saved day: 2020-03-23
28 saved day: 2020-03-22
29 saved day: 2020-03-21
30 saved day: 2020-03-20
31 saved day: 2020-03-19
32 saved day: 2020-03-18
33 saved day: 2020-03-17
34 saved day: 2020-03-16
35 saved day: 2020-03-15
36 saved day: 2020-03-14
37 saved day: 2020-03-13
38 saved day: 2020-03-12
39 saved day: 2020-03-11
40 saved day: 2020-03-10
41 saved day: 2020-03-09
42 saved day: 2020-03-08
43 saved day: 2020-03-07
44 saved day: 2020-03-06
45 saved day: 2020-03-05
46 saved day: 2020-03-04
47 saved day: 2020-03-03
48 saved day: 2020-03-02
49 saved day: 2020-03-01
50 saved day: 2020-02-29
51 saved day: 2020-02-28
52 saved day: 2020-02-27
53 saved day: 2020-02-26
54 saved day: 2020-02-25
55 saved day: 2020-02-24


In [29]:
len(data)

42725

In [28]:
data.drop_duplicates(inplace=True)

In [30]:
data.reset_index(drop=True, inplace=True)

In [36]:
data.to_csv('data.csv', index=None)